In [2]:
from dotenv import load_dotenv
load_dotenv()
from openai import OpenAI
client = OpenAI()

In [13]:
import os
import sys
import base64
import json
import pandas as pd

In [9]:
root_directory = os.path.dirname(os.path.dirname(os.getcwd()))
sys.path.append(os.path.join(root_directory,"src"))

import gptocr

In [15]:
import importlib
importlib.reload(gptocr)

<module 'gptocr' from 'c:\\Users\\kkoaz\\Documents\\projects\\product\\timetable-detect\\timetable-detect\\src\\gptocr.py'>

In [18]:
file_name = "20231226_TOKYO-GIRLS-GIRLS.jpeg"
prompt_user = "この画像のタイムテーブルをデータ化して"
return_json = gptocr.getocr_taiban_filename(file_name, prompt_user)
return_json

{'ライブ名': 'TOKYO GIRLS GIRLS',
 '会場名': '品川インターシティホール',
 '日付': '2023-12-26',
 'タイムテーブル': [{'グループ名': '8月のエウラルベ',
   'ライブステージ': [{'from': '8:20', 'to': '8:30'}],
   '特典会': [{'from': '8:40', 'to': '9:40', 'ブース': 'A'}]},
  {'グループ名': 'ポンコツコンポ',
   'ライブステージ': [{'from': '8:30', 'to': '8:40'}],
   '特典会': [{'from': '8:50', 'to': '9:50', 'ブース': 'B'}]},
  {'グループ名': 'MiX BeRRY',
   'ライブステージ': [{'from': '8:40', 'to': '8:50'}],
   '特典会': [{'from': '9:00', 'to': '10:00', 'ブース': 'C'}]},
  {'グループ名': 'make mie',
   'ライブステージ': [{'from': '8:50', 'to': '9:00'}],
   '特典会': [{'from': '9:10', 'to': '10:10', 'ブース': 'D'}]},
  {'グループ名': 'RINCENT#',
   'ライブステージ': [{'from': '9:00', 'to': '9:10'}],
   '特典会': [{'from': '9:20', 'to': '10:20', 'ブース': 'E'}]},
  {'グループ名': '天使にはなれない',
   'ライブステージ': [{'from': '9:10', 'to': '9:20'}],
   '特典会': [{'from': '9:30', 'to': '10:30', 'ブース': 'F'}]},
  {'グループ名': 'アルテミスの翼',
   'ライブステージ': [{'from': '9:20', 'to': '9:35'}],
   '特典会': [{'from': '9:45', 'to': '10:45', 'ブース': 'A'}]},
  {'グルー

In [28]:
system_prompt_taiban = """あなたは日本語に長けていて、また日本のアイドルシーンに詳しい専門家です。
与えられる画像は日本のアイドルの対バンライブについてのタイムテーブルです。
そこから情報を読み取り、
・ライブの名前
・会場の名前
・日付
・タイムテーブル（出演するアイドルの一覧とその出演時間）
をJSON形式で出力してください。

#ライブステージと特典会について
各アイドルは「ライブステージ」と「特典会」の2種類を行う可能性があります。
ライブステージの時間と合わせて、特典会の時間も出力してください。
ユーザーから「特典会無し」と指示された場合のみ、特典会の情報は出力せずライブステージの時間のみを出力してください。

#ライブステージについて
ライブステージは一つのアイドルグループが複数回出演する場合があります。
属性として「開始時間(from)」「終了時間(to)」を持ちます。

#特典会について
特典会は一つのアイドルグループが複数回行う場合があります。
属性として「開始時間(from)」「終了時間(to)」「ブース」を持ちます。
##特典会のブースについて
ブースとは、特典会の実施場所のことで、数字やアルファベット、日本語等で示されている場合があります。
時間の脇にアルファベット1文字が記載されている場合はそれをブースとして採用してください。
特に表記が見当たらない場合には、ブースは「なし」としてください。
##終演後特典会について
特典会は「終演後物販」「終演後特典会」といった形で、ライブ終了後に複数のグループが同時に行う場合があります。
特典会の時刻が画像からはわかりにくいグループは、一律の終演後特典会で行われる場合が多いです。
終演後特典会は時刻が明記されていない場合がありますが、
その場合は最後のライブステージの終了時刻を特典会の開始時刻として、そこから60分で計算してください。
時刻が明記されている場合は、終演後特典会であっても時間を60分ではなく明記されている時間を採用してください。

#ライブステージと特典会の判別方法
ライブステージの時間より、特典会の時間のほうが長い傾向にあります。
ライブステージはだいたい20分～30分程度であることが多く、短い場合は10分、長い場合でも40分に収まることが多いです。
特典会は60分前後の場合が多いです。

#出力形式
以下の様なJSON形式で出力してください。
{
    "ライブ名":"TOKYO GIRLS GIRLS"
    ,"会場名":"ZeppDiverCity"
    ,"日付":"2024-08-03"
    ,"タイムテーブル":
    [
        {
            "グループ名":"アイドルグループA"
            ,"ライブステージ":[
            {
                "from":"10:00"
                ,"to":"10:20"
            },
            ]
            ,"特典会":[
            {
                "from":"10:30"
                ,"to":"11:30"
                ,"ブース":"A"
            },
            ]
        },
        {
            "グループ名":"架空のアイドル"
            ,"ライブステージ":[
            {
                "from":"10:25"
                ,"to":"10:45"
            },
            ]
            ,"特典会":[
            {
                "from":"10:55"
                ,"to":"11:55"
                ,"ブース":"B"
            },
            ]
        }
    ]
}
"""

In [29]:
#フェスの場合のJSONの持ち方を変える？アイドルごとではなくステージごとにするとか
system_prompt_fes = """あなたは日本語に長けていて、また日本のアイドルシーンに詳しい専門家です。
与えられる画像は日本のアイドルフェスについてのタイムテーブルです。
そこから情報を読み取り、
・ライブの名前
・会場の名前
・日付
・タイムテーブル（出演するアイドルの一覧とその出演時間）
をJSON形式で出力してください。

#ライブステージと特典会について
各アイドルは「ライブステージ」と「特典会」の2種類を行う可能性があります。
ライブステージの時間と合わせて、特典会の時間も出力してください。
ユーザーから「特典会無し」と指示された場合のみ、特典会の情報は出力せずライブステージの時間のみを出力してください。

#ライブステージについて
ライブステージは一つのアイドルグループが複数回出演する場合があります。
ステージの種類が複数あり、それらが同時並行で進行する場合があります。
複数のステージがある場合、それぞれのステージには名前がついていることが一般的です。
属性として「開始時間(from)」「終了時間(to)」「ステージ名」を持ちます。

#特典会について
特典会は一つのアイドルグループが複数回行う場合があります。
属性として「開始時間(from)」「終了時間(to)」「ブース」を持ちます。
##特典会のブースについて
ブースとは、特典会の実施場所のことで、数字やアルファベット、日本語等で示されている場合があります。
時間の脇にアルファベット1文字が記載されている場合はそれをブースとして採用してください。
特に表記が見当たらない場合には、ブースは「なし」としてください。
##終演後特典会について
特典会は「終演後物販」「終演後特典会」といった形で、ライブ終了後に複数のグループが同時に行う場合があります。
特典会の時刻が画像からはわかりにくいグループは、一律の終演後特典会で行われる場合が多いです。
終演後特典会は時刻が明記されていない場合がありますが、
その場合は最後のライブステージの終了時刻を特典会の開始時刻として、そこから60分で計算してください。
時刻が明記されている場合は、終演後特典会であっても時間を60分ではなく明記されている時間を採用してください。

#ライブステージと特典会の判別方法
ライブステージの時間より、特典会の時間のほうが長い傾向にあります。
ライブステージはだいたい20分～30分程度であることが多く、短い場合は10分、長い場合でも40分に収まることが多いです。
特典会は60分前後の場合が多いです。

#出力形式
以下の様なJSON形式で出力してください。
{
    "ライブ名":"TOKYO IDOL FESTIVAL2024"
    ,"会場名":"ZeppDiverCity"
    ,"日付":"2024-08-03"
    ,"タイムテーブル":
    [
        {
            "グループ名":"アイドルグループA"
            ,"ライブステージ":[
            {
                "from":"10:00"
                ,"to":"10:20"
                ,"ステージ名":"なし"
            },
            ]
            ,"特典会":[
            {
                "from":"10:30"
                ,"to":"11:30"
                ,"ブース":"A"
            },
            ]
        },
        {
            "グループ名":"架空のアイドル"
            ,"ライブステージ":[
            {
                "from":"10:25"
                ,"to":"10:45"
                ,"ステージ名":"なし"
            },
            ]
            ,"特典会":[
            {
                "from":"10:55"
                ,"to":"11:55"
                ,"ブース":"B"
            },
            ]
        }
    ]
}
"""

In [18]:
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

In [45]:
def timetable_detect(image_path, fes_flag=False, user_prompt = "このライブの情報を教えて"):
    if fes_flag:
        system_prompt = system_prompt_fes
    else:
        system_prompt = system_prompt_taiban
    
    base64_image = encode_image(image_path)

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "system",
                "content": system_prompt
            },
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": user_prompt},
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{base64_image}",
                        },
                    },
                ],
            }
        ],
        response_format={"type": "json_object"},
        max_tokens=4096
    )

    return response

In [37]:
doc_folder = os.getcwd()+"/../../data/"
image_path = doc_folder + "timetable_sample/20231210_iCON-DOLL-LOUNGE.jpeg"
response = timetable_detect(image_path)
json.loads(response.choices[0].message.content)

{'ライブ名': 'iCON DOLL LOUNGE 2023〜YEARS END SPECIAL 3MAN〜',
 '会場名': 'なし',
 '日付': '2023-12-10',
 'タイムテーブル': [{'グループ名': 'Mirror,Mirror',
   'ライブステージ': [{'from': '13:00', 'to': '13:35'}],
   '特典会': [{'from': '14:45', 'to': '15:45', 'ブース': 'なし'}]},
  {'グループ名': 'THE ORCHESTRA TOKYO',
   'ライブステージ': [{'from': '13:35', 'to': '14:10'}],
   '特典会': [{'from': '14:45', 'to': '15:45', 'ブース': 'なし'}]},
  {'グループ名': 'Ringwanderung',
   'ライブステージ': [{'from': '14:10', 'to': '14:45'}],
   '特典会': [{'from': '14:45', 'to': '15:45', 'ブース': 'なし'}]}]}

In [32]:
image_path = doc_folder + "timetable_sample/20231209_MUD-PARTY.jpeg"
response = timetable_detect(image_path)
json.loads(response.choices[0].message.content)

{'ライブ名': 'MUD PARTY vol.3',
 '会場名': 'SHIROKANE TAKANAWA SELENE b2',
 '日付': '2023-12-09',
 'タイムテーブル': [{'グループ名': 'HACK',
   'ライブステージ': [{'from': '10:00', 'to': '10:20'}],
   '特典会': [{'from': '10:35', 'to': '11:35', 'ブース': 'A'}]},
  {'グループ名': 'アキストゼネコ',
   'ライブステージ': [{'from': '10:20', 'to': '10:40'}],
   '特典会': [{'from': '10:55', 'to': '11:55', 'ブース': 'B'}]},
  {'グループ名': 'TELLMIT',
   'ライブステージ': [{'from': '10:40', 'to': '11:00'}],
   '特典会': [{'from': '11:15', 'to': '12:15', 'ブース': 'A'}]},
  {'グループ名': 'さよならステイチューン',
   'ライブステージ': [{'from': '11:00', 'to': '11:20'}],
   '特典会': [{'from': '11:35', 'to': '12:35', 'ブース': 'C'}]},
  {'グループ名': 'ベンジャス！',
   'ライブステージ': [{'from': '11:20', 'to': '11:40'}],
   '特典会': [{'from': '11:40', 'to': '12:40', 'ブース': 'B'}]},
  {'グループ名': 'zanka',
   'ライブステージ': [{'from': '11:40', 'to': '12:00'}],
   '特典会': [{'from': '12:15', 'to': '13:15', 'ブース': 'A'}]},
  {'グループ名': 'MEWM',
   'ライブステージ': [{'from': '12:05', 'to': '12:30'}],
   '特典会': [{'from': '12:45', 'to': '13:4

In [33]:
image_path = doc_folder + "timetable_sample/20231226_TOKYO-GIRLS-GIRLS.jpeg"
response = timetable_detect(image_path)
json.loads(response.choices[0].message.content)

{'ライブ名': 'TOKYO GIRLS GIRLS',
 '会場名': '品川インターシティホール',
 '日付': '2023-12-26',
 'タイムテーブル': [{'グループ名': '8月のエヴァラベ',
   'ライブステージ': [{'from': '8:20', 'to': '8:30'}],
   '特典会': [{'from': '8:40', 'to': '9:40', 'ブース': 'A'}]},
  {'グループ名': 'ボンコツコンポ',
   'ライブステージ': [{'from': '8:30', 'to': '8:40'}],
   '特典会': [{'from': '8:50', 'to': '9:50', 'ブース': 'B'}]},
  {'グループ名': 'MiX BeRRY',
   'ライブステージ': [{'from': '8:40', 'to': '8:50'}],
   '特典会': [{'from': '9:00', 'to': '10:00', 'ブース': 'C'}]},
  {'グループ名': 'make mie',
   'ライブステージ': [{'from': '8:50', 'to': '9:00'}],
   '特典会': [{'from': '9:10', 'to': '10:10', 'ブース': 'D'}]},
  {'グループ名': 'RiNCENT#',
   'ライブステージ': [{'from': '9:00', 'to': '9:10'}],
   '特典会': [{'from': '9:20', 'to': '10:20', 'ブース': 'D'}]},
  {'グループ名': '天使にはなれない',
   'ライブステージ': [{'from': '9:10', 'to': '9:20'}],
   '特典会': [{'from': '9:30', 'to': '10:30', 'ブース': 'A'}]},
  {'グループ名': 'アルテミスの翼',
   'ライブステージ': [{'from': '9:20', 'to': '9:35'}],
   '特典会': [{'from': '9:50', 'to': '10:50', 'ブース': 'B'}]},
  {

In [34]:
image_path = doc_folder + "timetable_sample/20231203_TODOROCK-IDOL-FESTIVAL.jpeg"
response = timetable_detect(image_path, True)
json.loads(response.choices[0].message.content)

{'ライブ名': 'TODOROKI!! IDOL FESTIVAL 2023',
 '会場名': 'KAWASAKI TODOROKI ARENA',
 '日付': '2023-12-03',
 'タイムテーブル': [{'グループ名': 'yosugala',
   'ライブステージ': [{'from': '10:00', 'to': '10:20', 'ステージ名': 'アリーナ'}],
   '特典会': [{'from': '10:30', 'to': '11:30', 'ブース': 'B'}]},
  {'グループ名': 'Quubi',
   'ライブステージ': [{'from': '10:40', 'to': '11:00', 'ステージ名': 'アリーナ'}],
   '特典会': [{'from': '11:10', 'to': '12:10', 'ブース': 'B'}]},
  {'グループ名': 'シュレーディンガーの犬',
   'ライブステージ': [{'from': '11:10', 'to': '11:30', 'ステージ名': 'アリーナ'}],
   '特典会': [{'from': '11:35', 'to': '12:35', 'ブース': 'A'}]},
  {'グループ名': 'MIGMA SHELTER',
   'ライブステージ': [{'from': '11:40', 'to': '12:00', 'ステージ名': 'アリーナ'}],
   '特典会': [{'from': '12:10', 'to': '13:10', 'ブース': 'B'}]},
  {'グループ名': 'Kolokol',
   'ライブステージ': [{'from': '12:10', 'to': '12:30', 'ステージ名': 'アリーナ'}],
   '特典会': [{'from': '12:35', 'to': '13:35', 'ブース': 'C'}]},
  {'グループ名': 'Ringwanderung',
   'ライブステージ': [{'from': '12:50', 'to': '13:10', 'ステージ名': 'アリーナ'}],
   '特典会': [{'from': '13:15', 'to': '14:15

In [46]:
image_path = doc_folder + "timetable_sample/20240503_JCIF-LIVE.jpeg"
response = timetable_detect(image_path, True, "このライブの情報を教えて。ただし特典会情報はこの画像にはないのでライブステージのみを出力して")
json.loads(response.choices[0].message.content)

{'ライブ名': 'JAPAN CENTRAL IDOL FESTIVAL 2024',
 '会場名': 'AICHI SKY EXPO',
 '日付': '2024-05-03',
 'タイムテーブル': [{'グループ名': 'THE ENCORE',
   'ライブステージ': [{'from': '09:30', 'to': '09:50', 'ステージ名': 'CENTRAL STAGE'},
    {'from': '17:30', 'to': '17:50', 'ステージ名': 'SKY STAGE'}]},
  {'グループ名': 'アンター・ビースマイナー',
   'ライブステージ': [{'from': '09:50', 'to': '10:10', 'ステージ名': 'CENTRAL STAGE'}]},
  {'グループ名': 'dela',
   'ライブステージ': [{'from': '10:10', 'to': '10:30', 'ステージ名': 'CENTRAL STAGE'},
    {'from': '16:10', 'to': '16:30', 'ステージ名': 'OCEAN STAGE'}]},
  {'グループ名': '感覚ロヂック',
   'ライブステージ': [{'from': '10:50', 'to': '11:10', 'ステージ名': 'CENTRAL STAGE'}]},
  {'グループ名': '虹の飛行少女',
   'ライブステージ': [{'from': '11:10', 'to': '11:30', 'ステージ名': 'CENTRAL STAGE'},
    {'from': '16:30', 'to': '16:50', 'ステージ名': 'CORAL STAGE'}]},
  {'グループ名': 'なないろの雨',
   'ライブステージ': [{'from': '11:30', 'to': '11:50', 'ステージ名': 'CENTRAL STAGE'}]},
  {'グループ名': 'ワンダーウィード 天',
   'ライブステージ': [{'from': '11:50', 'to': '12:10', 'ステージ名': 'CENTRAL STAGE'}]},
  {'グループ

In [40]:
print(response.choices[0].message.content)

{
    "ライブ名":"JAPAN CENTRAL IDOL FESTIVAL 2024",
    "会場名":"AICHI SKY EXPO",
    "日付":"2024-05-03",
    "タイムテーブル":
    [
        {
            "グループ名":"THE ENCORE",
            "ライブステージ":[
                {
                    "from":"09:30",
                    "to":"09:50",
                    "ステージ名":"CENTRAL STAGE"
                }
            ]
        },
        {
            "グループ名":"アンチーテーゼ",
            "ライブステージ":[
                {
                    "from":"09:50",
                    "to":"10:10",
                    "ステージ名":"CENTRAL STAGE"
                }
            ]
        },
        {
            "グループ名":"dela",
            "ライブステージ":[
                {
                    "from":"10:10",
                    "to":"10:30",
                    "ステージ名":"CENTRAL STAGE"
                }
            ]
        },
        {
            "グループ名":"藍那サノ@drop tune",
            "ライブステージ":[
                {
                    "from":"10:30",
                    "to":"10:50",

In [9]:
#読み取ったJSONの加工
filename = "20231226_TOKYO-GIRLS-GIRLS"
json_path = "../../data/output/timetable_taiban/{file_name}.json".format(file_name=filename.split(".")[0])
with open(json_path,"r",encoding = "utf8") as f:
    return_json = json.load(f)
return_json

{'ライブ名': 'TOKYO GIRLS GIRLS',
 '会場名': '品川インターシティホール',
 '日付': '2023-12-26',
 'タイムテーブル': [{'グループ名': '8月のエウラルベ',
   'ライブステージ': {'from': '8:20', 'to': '8:30'},
   '特典会': [{'from': '8:40', 'to': '9:40', 'ブース': 'A'}]},
  {'グループ名': 'ポンコツコンポ',
   'ライブステージ': {'from': '8:30', 'to': '8:40'},
   '特典会': [{'from': '8:50', 'to': '9:50', 'ブース': 'B'}]},
  {'グループ名': 'MiX BeRRY',
   'ライブステージ': {'from': '8:40', 'to': '8:50'},
   '特典会': [{'from': '9:00', 'to': '10:00', 'ブース': 'C'}]},
  {'グループ名': 'make mie',
   'ライブステージ': {'from': '8:50', 'to': '9:00'},
   '特典会': [{'from': '9:10', 'to': '10:10', 'ブース': 'D'}]},
  {'グループ名': 'RiNCENT#',
   'ライブステージ': {'from': '9:00', 'to': '9:10'},
   '特典会': [{'from': '9:20', 'to': '10:20', 'ブース': 'E'}]},
  {'グループ名': '天使にはなれない',
   'ライブステージ': {'from': '9:10', 'to': '9:20'},
   '特典会': [{'from': '9:30', 'to': '10:30', 'ブース': 'A'}]},
  {'グループ名': 'アルテミスの翼',
   'ライブステージ': {'from': '9:20', 'to': '9:35'},
   '特典会': [{'from': '9:50', 'to': '10:50', 'ブース': 'B'}]},
  {'グループ名': 'NiLULOCK

In [28]:
df_timetable = []
for item in return_json["タイムテーブル"]:
    # ライブステージの時間を取得
    try:
        live_stage_from = item['ライブステージ']['from']
    except KeyError:
        live_stage_from = ""
    try:
        live_stage_to = item['ライブステージ']['to']
    except KeyError:
        live_stage_to = ""
    
    # 特典会の情報を処理
    for meeting in item['特典会']:
        try:
            meeting_from = meeting['from']
        except KeyError:
            meeting_from = ""
        try:
            meeting_to = meeting['to']
        except KeyError:
            meeting_to = ""
        try:
            booth = meeting['ブース']
        except KeyError:
            booth = ""
        
        # DataFrameに行を追加
        df_timetable.append({
            'グループ名': item['グループ名'],
            'ライブステージ_from': live_stage_from,
            'ライブステージ_to': live_stage_to,
            '特典会_from': meeting_from,
            '特典会_to': meeting_to,
            'ブース': booth
        })

# 列名を設定
df_timetable = pd.DataFrame(df_timetable,columns=['グループ名', 'ライブステージ_from', 'ライブステージ_to', '特典会_from', '特典会_to', 'ブース'])

# 結果の表示
display(df_timetable)

,グループ名,ライブステージ_from,ライブステージ_to,特典会_from,特典会_to,ブース
0,8月のエウラルベ,8:20,8:30,8:40,9:40,A
1,ポンコツコンポ,8:30,8:40,8:50,9:50,B
2,MiX BeRRY,8:40,8:50,9:00,10:00,C
3,make mie,8:50,9:00,9:10,10:10,D
4,RiNCENT#,9:00,9:10,9:20,10:20,E
5,天使にはなれない,9:10,9:20,9:30,10:30,A
6,アルテミスの翼,9:20,9:35,9:50,10:50,B
7,NiLULOCK,9:40,9:55,10:05,11:05,B
8,ZUTTOMOTTO,9:55,10:10,10:20,11:20,C
9,alma,10:10,10:25,10:35,11:35,D


In [8]:
data_sorted_index = 
for i, group_data in enumerate(return_json["タイムテーブル"]):
    print(group_data["ライブステージ"][0]["from"])

8:20
8:30
8:40
8:50
9:00
9:10
9:20
9:40
9:55
10:10
10:25
10:40
11:00
11:20
11:45
12:05
12:25
12:45
13:05
13:25
13:45
14:10
14:30
14:50
15:10
15:30
15:50
16:10
16:35
16:55
17:15
17:35
18:00
18:40
19:00
19:20
19:45
20:10
20:30
21:00
